In [ ]:
import torch

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/name_gender_dataset.csv'
DATASET = pd.read_csv(PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATASET.head()

,Name,Gender,Count,Probability
0,James,M,5304407,0.014517
1,John,M,5260831,0.014398
2,Robert,M,4970386,0.013603
3,Michael,M,4579950,0.012534
4,William,M,4226608,0.011567


In [ ]:
DATASET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147269 entries, 0 to 147268
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Name         147269 non-null  object 
 1   Gender       147269 non-null  object 
 2   Count        147269 non-null  int64  
 3   Probability  147269 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.5+ MB


In [ ]:
DATASET.describe()

,Count,Probability
count,1.472690e+05,1.472690e+05
mean,2.481161e+03,6.790295e-06
std,4.645472e+04,1.271345e-04
min,1.000000e+00,2.736740e-09
25%,5.000000e+00,1.368370e-08
50%,1.700000e+01,4.652460e-08
75%,1.320000e+02,3.612500e-07
max,5.304407e+06,1.451679e-02


In [ ]:
DATASET.value_counts

Name      Gender  Count  Probability 
A         F       2      5.473480e-09    1
Nashira   F       273    7.471300e-07    1
Nashiem   M       19     5.199810e-08    1
Nashika   F       75     2.052560e-07    1
Nashim    M       1      2.736740e-09    1
                                        ..
Gundeep   M       1      2.736740e-09    1
          F       1      2.736740e-09    1
Gundaris  M       1      2.736740e-09    1
Gunda     F       118    3.229350e-07    1
Zzyzx     M       10     2.736740e-08    1
Length: 147269, dtype: int64

In [ ]:
SRC = DATASET.Name.values.tolist()       
TRG = DATASET.Gender.values.tolist()       

In [ ]:
print(SRC)

['James', 'John', 'Robert', 'Michael', 'William', 'Mary', 'David', 'Joseph', 'Richard', 'Charles', 'Thomas', 'Christopher', 'Daniel', 'Matthew', 'Elizabeth', 'Patricia', 'Jennifer', 'Anthony', 'George', 'Linda', 'Barbara', 'Donald', 'Paul', 'Mark', 'Andrew', 'Steven', 'Kenneth', 'Edward', 'Joshua', 'Margaret', 'Brian', 'Kevin', 'Jessica', 'Sarah', 'Susan', 'Timothy', 'Dorothy', 'Jason', 'Ronald', 'Helen', 'Ryan', 'Jeffrey', 'Karen', 'Nancy', 'Betty', 'Lisa', 'Jacob', 'Nicholas', 'Ashley', 'Eric', 'Frank', 'Gary', 'Anna', 'Stephen', 'Jonathan', 'Sandra', 'Emily', 'Amanda', 'Kimberly', 'Michelle', 'Donna', 'Justin', 'Laura', 'Ruth', 'Carol', 'Brandon', 'Larry', 'Scott', 'Melissa', 'Stephanie', 'Benjamin', 'Raymond', 'Samuel', 'Rebecca', 'Deborah', 'Gregory', 'Sharon', 'Kathleen', 'Amy', 'Cynthia', 'Alexander', 'Patrick', 'Jack', 'Henry', 'Angela', 'Shirley', 'Emma', 'Catherine', 'Katherine', 'Virginia', 'Nicole', 'Dennis', 'Walter', 'Tyler', 'Peter', 'Brenda', 'Aaron', 'Jerry', 'Christin

In [ ]:
from sklearn.model_selection import train_test_split

TRAIN_DATA, TEST_DATA, TRAIN_LABEL, TEST_LABEL = train_test_split(SRC, TRG, test_size=0.2, random_state=42, shuffle=True)
TRAIN_DATA, VALID_DATA, TRAIN_LABEL, VALID_LABEL = train_test_split(TRAIN_DATA, TRAIN_LABEL, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
print(TRAIN_DATA)

['Tilo', 'Zackery', 'Freland', 'Indiara', 'Evins', 'Gudrun', 'Jovanie', 'Deveshi', 'Isabeth', 'Makaiah', 'Jashelle', 'Namra', 'Zacchary', 'Divam', 'Reynell', 'Gena-Maria', 'Nykhia', 'Marriann', 'Sallyanne-Marie', 'Aliana', 'Berlena', 'Fawne', 'Torr', 'Maydelyn', 'Josslin', 'Sacora', 'Folad', 'Niquan', 'Vernicia', 'Shennell', 'Jacquee', 'Donoven', 'Azzan', 'Amiyra', 'Leoner', 'Dimytri', 'Krzystof', 'Chedrick', 'Brockston', 'Denella', 'Kira-Lea', 'Makalah', 'Sakila', 'Embyr', 'Mahoghany', 'Reace', 'Dosie', 'Daneya', 'Port', 'Smt.Lokesh', 'Freylin', 'Sebastijan', 'Elizabethanne', 'Anahit', 'Lawence', 'Roja', 'Marquist', 'Kaynon', 'Mal', 'Safira', 'Minelly', 'Zarielle', 'Nicoly', 'Kanesha', 'Gregeory', 'Tjustin', 'Lannie', 'Johaun', 'Sanja-Lana', 'Sandeep', 'Raymesha', 'Judy-Ann', 'Lauren-Marie', 'Tanene', 'Vanesa', 'Kaynen', 'Ledell', 'Jon-Jacobs', 'Aldie', 'Brookleigh', 'Janeily', 'Olyve', 'Marshanti', 'Shannondoah', 'Tyzaeiha', 'Jerahmy', 'Ayverie', 'Caylan', 'Terrense', 'Puneet', 'Shar

In [ ]:
'''convert to the char '''

char_set = set()

for name in TRAIN_DATA:
    for ch in list(name):
        char_set.add(ch.lower())

print(char_set)

{'j', 'à', '5', '0', 'ö', '…', 'f', 'u', 'z', ';', '8', 'w', 'b', 'œ', 'q', 'e', '?', 'y', 'm', 'k', 's', '-', 'x', '(', ',', 'l', 'p', '1', 'i', 'c', 'v', '@', '.', '9', ')', '¡', 'a', 'o', '[', '/', 't', 'r', 'n', "'", '7', 'g', 'h', 'd'}


In [ ]:
CHAR2INDEX = {char: indx + 2 for indx, char in enumerate(char_set)}
CHAR2INDEX['<UNK>'] = 1 # for OOV
CHAR2INDEX['<PAD>'] = 0 # for PADDING


INDEX2CHAR = {indx + 2: char for indx, char in enumerate(char_set)}
INDEX2CHAR[1] = '<UNK>' # for OOV
INDEX2CHAR[0] = '<PAD>' # for PADDING

In [ ]:
print(CHAR2INDEX)
print(INDEX2CHAR)

{'j': 2, 'à': 3, '5': 4, '0': 5, 'ö': 6, '…': 7, 'f': 8, 'u': 9, 'z': 10, ';': 11, '8': 12, 'w': 13, 'b': 14, 'œ': 15, 'q': 16, 'e': 17, '?': 18, 'y': 19, 'm': 20, 'k': 21, 's': 22, '-': 23, 'x': 24, '(': 25, ',': 26, 'l': 27, 'p': 28, '1': 29, 'i': 30, 'c': 31, 'v': 32, '@': 33, '.': 34, '9': 35, ')': 36, '¡': 37, 'a': 38, 'o': 39, '[': 40, '/': 41, 't': 42, 'r': 43, 'n': 44, "'": 45, '7': 46, 'g': 47, 'h': 48, 'd': 49, '<UNK>': 1, '<PAD>': 0}
{2: 'j', 3: 'à', 4: '5', 5: '0', 6: 'ö', 7: '…', 8: 'f', 9: 'u', 10: 'z', 11: ';', 12: '8', 13: 'w', 14: 'b', 15: 'œ', 16: 'q', 17: 'e', 18: '?', 19: 'y', 20: 'm', 21: 'k', 22: 's', 23: '-', 24: 'x', 25: '(', 26: ',', 27: 'l', 28: 'p', 29: '1', 30: 'i', 31: 'c', 32: 'v', 33: '@', 34: '.', 35: '9', 36: ')', 37: '¡', 38: 'a', 39: 'o', 40: '[', 41: '/', 42: 't', 43: 'r', 44: 'n', 45: "'", 46: '7', 47: 'g', 48: 'h', 49: 'd', 1: '<UNK>', 0: '<PAD>'}


In [ ]:
LABEL2INDEX = {'M':0,'F':1}
INDEX2LABEL = {0:'M',1:'F'}

In [ ]:
print(LABEL2INDEX)
print(INDEX2LABEL)

{'M': 0, 'F': 1}
{0: 'M', 1: 'F'}


In [ ]:
# set device to CUDA if available, else to CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', DEVICE)

Device: cuda


In [ ]:
from torch.utils.data import Dataset, DataLoader

class DATASET(Dataset):
  def __init__(self, names, genders,mapping_data, mapping_label):
    self.names= names
    self.names = [indexing(name,mapping_data) for name in names]
    self.genders = genders
    self.genders = [mapping_label[gender] for gender in self.genders]
    self.length = len(self.names)
  def __getitem__(self, item):
    return self.names[item], self.genders[item]
  def __len__(self):
    return self.length

In [ ]:
def indexing(name,mapping_data):
  return [mapping_data[char.lower()] if char.lower() in mapping_data else mapping_data['<UNK>'] for char in name]

In [ ]:
DATASET_TRAIN = DATASET(names=TRAIN_DATA, genders=TRAIN_LABEL, mapping_data=CHAR2INDEX,
                              mapping_label=LABEL2INDEX)
DATASET_VALID = DATASET(names=VALID_DATA, genders=VALID_LABEL, mapping_data=CHAR2INDEX,
                              mapping_label=LABEL2INDEX)
DATASET_TEST = DATASET(names=TEST_DATA, genders=TEST_LABEL, mapping_data=CHAR2INDEX,
                              mapping_label=LABEL2INDEX)

In [ ]:
(DATASET_TRAIN.names)

[[42, 30, 27, 39],
 [10, 38, 31, 21, 17, 43, 19],
 [8, 43, 17, 27, 38, 44, 49],
 [30, 44, 49, 30, 38, 43, 38],
 [17, 32, 30, 44, 22],
 [47, 9, 49, 43, 9, 44],
 [2, 39, 32, 38, 44, 30, 17],
 [49, 17, 32, 17, 22, 48, 30],
 [30, 22, 38, 14, 17, 42, 48],
 [20, 38, 21, 38, 30, 38, 48],
 [2, 38, 22, 48, 17, 27, 27, 17],
 [44, 38, 20, 43, 38],
 [10, 38, 31, 31, 48, 38, 43, 19],
 [49, 30, 32, 38, 20],
 [43, 17, 19, 44, 17, 27, 27],
 [47, 17, 44, 38, 23, 20, 38, 43, 30, 38],
 [44, 19, 21, 48, 30, 38],
 [20, 38, 43, 43, 30, 38, 44, 44],
 [22, 38, 27, 27, 19, 38, 44, 44, 17, 23, 20, 38, 43, 30, 17],
 [38, 27, 30, 38, 44, 38],
 [14, 17, 43, 27, 17, 44, 38],
 [8, 38, 13, 44, 17],
 [42, 39, 43, 43],
 [20, 38, 19, 49, 17, 27, 19, 44],
 [2, 39, 22, 22, 27, 30, 44],
 [22, 38, 31, 39, 43, 38],
 [8, 39, 27, 38, 49],
 [44, 30, 16, 9, 38, 44],
 [32, 17, 43, 44, 30, 31, 30, 38],
 [22, 48, 17, 44, 44, 17, 27, 27],
 [2, 38, 31, 16, 9, 17, 17],
 [49, 39, 44, 39, 32, 17, 44],
 [38, 10, 10, 38, 44],
 [38, 20, 30

In [ ]:
print(DATASET_TRAIN.genders)

[0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 

In [ ]:
from torch import nn
from torch.nn.utils.rnn import pad_sequence

def pad_collate(batch):
  (data, label) = zip(*batch)
  data = [torch.tensor(i, dtype=torch.int) for i in data]
  label = torch.tensor(label, dtype=torch.float)

  data = pad_sequence(data, batch_first=True, padding_value=0)
  data = torch.tensor(data, dtype=torch.int)

  return data, label

In [ ]:
""" Hyper-Parameter Definition """

BATCH_SIZE = 32
EPOCH_SIZE = 30

INPUT_SIZE = len(CHAR2INDEX)
HIDDEN_SIZE = 256
OUTPUT_DIM = 1
N_LAYES = 1
LEARNING_RATE = 0.001


DATALOADER_TRAIN = DataLoader(dataset=DATASET_TRAIN,
                              batch_size=BATCH_SIZE,
                              num_workers=4,
                              collate_fn=pad_collate)

DATALOADER_TEST = DataLoader(dataset=DATASET_TEST,
                              batch_size=BATCH_SIZE,
                              num_workers=4,
                              collate_fn=pad_collate)

DATALOADER_VALID = DataLoader(dataset=DATASET_VALID,
                              batch_size=BATCH_SIZE,
                              num_workers=4,
                              collate_fn=pad_collate)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
iter(DATALOADER_TRAIN).next()


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather t

(tensor([[42, 30, 27, 39,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [10, 38, 31, 21, 17, 43, 19,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 8, 43, 17, 27, 38, 44, 49,  0,  0,  0,  0,  0,  0,  0,  0],
         [30, 44, 49, 30, 38, 43, 38,  0,  0,  0,  0,  0,  0,  0,  0],
         [17, 32, 30, 44, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [47,  9, 49, 43,  9, 44,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 2, 39, 32, 38, 44, 30, 17,  0,  0,  0,  0,  0,  0,  0,  0],
         [49, 17, 32, 17, 22, 48, 30,  0,  0,  0,  0,  0,  0,  0,  0],
         [30, 22, 38, 14, 17, 42, 48,  0,  0,  0,  0,  0,  0,  0,  0],
         [20, 38, 21, 38, 30, 38, 48,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 2, 38, 22, 48, 17, 27, 27, 17,  0,  0,  0,  0,  0,  0,  0],
         [44, 38, 20, 43, 38,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [10, 38, 31, 31, 48, 38, 43, 19,  0,  0,  0,  0,  0,  0,  0],
         [49, 30, 32, 38, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      

In [ ]:
class MODEL(nn.Module):
  def __init__(self, input_size, hidden_size, output_dim, n_layes, batch_size):
    super().__init__()
    self.hidden_size=hidden_size
    self.output_dim=output_dim
    self.n_layes=n_layes
    self.batch_size=batch_size
    self.embedding= nn.Embedding(input_size,self.hidden_size)
    self.lstm=nn.LSTM(self.input_size, self.hidden_size, self.n_layers)
    self.fully_connected=nn.Linear(hidden_size,output_dim)

  
    def forward(self, input):
      input=input.t
      self.sent_len=input.size(0)

      # print(input.size())
      embedded = self.embedding(input)
      self.batch_size = embedded.size(1)
      output, (hidden, cell) = self.lstm(embedded)
